In [10]:
!git clone https://github.com/vohoaidanh/DIRE.git

Cloning into 'DIRE'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 76 (delta 18), reused 71 (delta 13), pack-reused 0
Receiving objects: 100% (76/76), 1.96 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [1]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt

--2024-04-24 05:24:15--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.179.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.179.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2211383297 (2.1G) [application/octet-stream]
Saving to: ‘256x256_diffusion_uncond.pt’

256x256_diffusion_u 100%[===================>]   2.06G  18.3MB/s    in 97s     

2024-04-24 05:25:52 (21.8 MB/s) - ‘256x256_diffusion_uncond.pt’ saved [2211383297/2211383297]



In [3]:
import os
os.makedirs('DIRE/models', exist_ok=True)
import shutil
shutil.move('/workspace/256x256_diffusion_uncond.pt', '/workspace/DIRE/models')


'/workspace/DIRE/models/256x256_diffusion_uncond.pt'

In [4]:
!pip install ftfy -q
!pip install natsort -q
!pip install tensorboardX -q
!pip install blobfile -q
#!pip install mpi4py -q
!pip install scipy

##############################

!apt update
!apt install unzip
!pip install pandas -q
!pip install scikit-image -q
!pip install opencv-python -q
!apt-get install -y libgl1-mesa-glx -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 63.9 MB/s eta 0:00:0000:0100:01


In [12]:
#@title Unzip dataset
#!cp /content/drive/MyDrive/DATASETS/RealFakeDB_tiny.zip -d /content
dataset_zip_file = '/workspace/RealFakeDB512s.zip' #@param {type:"string"}
unzip_to = '/workspace/DIRE/RealFakeDB512s' #@param {type:"string"}
!unzip -q $dataset_zip_file -d $unzip_to

replace /workspace/DIRE/RealFakeDB512s/test/0_real/000000356941.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [13]:
%cd /workspace/DIRE

/workspace/DIRE


#Process for test set
(We need to remove all lines related to multi-GPU configuration in compe_dire.py before run)

#Process for train set

In [ ]:
import os
image_list = []
extensions = ['.jpg', '.png', '.gif', '.bmp', '.webp']  # Add more extensions as needed
def getname(base_dir):
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if len(filenames) > 0:
            imgs = [filename for filename in filenames if any(ext in filename for ext in extensions)]
            image_list.extend(imgs)
    return image_list


In [ ]:
image_list=[]
image_list = getname('/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/dire_test')


In [ ]:
len(image_list)

992

In [ ]:
#@title Remove all items existed in $image_list before run
data_dir = '/workspace/DIRE/RealFakeDB512s' #@param {type:"string"}
c=0
for dirpath, dirnames, filenames in os.walk(data_dir):
    if len(filenames) >0:
        for file in filenames:
            if file in image_list:
                c+=1
                full_name = os.path.join(dirpath, file)
                os.remove(full_name)
                print('removed {}:{}:'.format(c,full_name))

samples = len(os.listdir(data_dir + '/0_real')) + len(os.listdir(data_dir + '/1_fake'))
print('num sample remain :', samples)

In [8]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# edit add wepb to        if "." in entry and ext.lower() in ["jpg", "jpeg", "png", "gif", "webp"]: in image_dataset.py gaussioan_diffusion
set_data = 'val'
class_name = '0_real'

recons_test = f'recons_{set_data}'
dire_test = f'dire_{set_data}'
num_samples = len(os.listdir( '/workspace/DIRE/RealFakeDB512s/' + set_data + '/' + class_name))
print('recons_test: ',recons_test)
print('dire_test: ', dire_test)
print('num_samples',num_samples)

!python guided-diffusion/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir   /workspace/DIRE/RealFakeDB512s/$set_data/$class_name \
--recons_dir  /workspace/DIRE/out_put/RealFakeDB512s/$recons_test/$class_name \
--dire_dir   /workspace/DIRE/out_put/RealFakeDB512s/$dire_test/$class_name \
--batch_size 1 --num_samples $num_samples  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder False

recons_test:  recons_val
dire_test:  dire_val
num_samples 3000
Logging to /workspace/DIRE/out_put/RealFakeDB512s/recons_val/0_real
Namespace(images_dir='/workspace/DIRE/RealFakeDB512s/val/0_real', recons_dir='/workspace/DIRE/out_put/RealFakeDB512s/recons_val/0_real', dire_dir='/workspace/DIRE/out_put/RealFakeDB512s/dire_val/0_real', clip_denoised=True, num_samples=3000, batch_size=1, use_ddim=True, model_path='models/256x256_diffusion_uncond.pt', real_step=0, continue_reverse=False, has_subfolder=False, image_size=256, num_channels=256, num_res_blocks=2, num_heads=4, num_heads_upsample=-1, num_head_channels=64, attention_resolutions='32,16,8', channel_mult='', dropout=0.1, class_cond=False, use_checkpoint=False, use_scale_shift_norm=True, resblock_updown=True, use_fp16=True, use_new_attention_order=False, learn_sigma=True, diffusion_steps=1000, noise_schedule='linear', timestep_respacing='ddim20', use_kl=False, predict_xstart=False, rescale_timesteps=False, rescale_learned_sigmas=False